In [1]:
!pip install segmentation_models_pytorch 
!pip install neptune
!pip install torchmetrics
!pip install albumentations 

In [2]:
import torch
import torch.nn.functional as F
from torch.optim import lr_scheduler
import numpy as np
import torch.nn as nn
import time
import copy
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from PIL import Image
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from segmentation_models_pytorch.encoders import get_preprocessing_fn
import segmentation_models_pytorch as smp
import neptune
from neptune.types import File

from torchmetrics.functional.classification import dice as calc_dice_score
from torchmetrics.classification import BinaryJaccardIndex

import albumentations as A
import cv2

In [3]:
# Define a custom dataset class
# Training params
BATCH_SIZE = 13
EPOCHS = 110
LEARNING_RATE = 0.0000367

# Model params
ENCODER_NAME = "resnet34"
ENCODER_WEIGHTS = "imagenet"

with open("/notebooks/Testing/NEPTUNE_API_TOKEN.txt", "r") as file:
    # Read the entire content of the file into a string
    token = file.read()

run = neptune.init_run(
    project="Kernel-bois/computer-vision",
    api_token=token,
)
run_id = run["sys/id"].fetch()

# Create the model
model = smp.Unet(
    encoder_name=ENCODER_NAME,           # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights=ENCODER_WEIGHTS,     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                        # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                            # model output channels (number of classes in your dataset)
    )


# Define optimizer
optimizer = optim.RMSprop(model.parameters(), lr=LEARNING_RATE)
#Scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

run_name = "MODEL-" + model.__class__.__name__ + ENCODER_NAME + str(run_id)

save_path = str(run_id) + "/"
os.makedirs(save_path)

# Proper directories
TRAIN_DATA_DIR = '/notebooks/image_segmentation/network/image_data_all/train'
VAL_DATA_DIR = '/notebooks/image_segmentation/network/image_data_all/val'

# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss

        return Dice_BCE



# Define loss function
# criterion = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=False)  # Binary dice Loss for binary segmentation
# criterion = smp.losses.SoftBCEWithLogitsLoss()  # Binary dice Loss for binary segmentation
criterion = DiceBCELoss()  # Binary dice Loss for binary segmentation
calc_iou = BinaryJaccardIndex().to(device)


params = {
    "MODEL": model.__class__.__name__,
    "BACKBONE": ENCODER_NAME,
    "ENCODER_WEIGHTS": ENCODER_WEIGHTS,
    "BATCH_SIZE": str(BATCH_SIZE),
    "EPOCHS": str(EPOCHS),
    "CRITERION": criterion.__class__.__name__,
    "OPTIMIZER": optimizer.__class__.__name__,
    "LEARNRATE": str(LEARNING_RATE),
    "MODEL_NAME": run_name,
}

run["params"] = params



/tmp/ipykernel_21830/2663222614.py:15: NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/
  run = neptune.init_run(


https://app.neptune.ai/Kernel-bois/computer-vision/e/CV-151


In [4]:
class SegmentationDataset(Dataset):
    def __init__(self, root_dir, transform = None, target_size = (992, 416)):
        self.root_dir = root_dir
        self.transform = transform
        self.target_size = target_size

        self.image_folder = os.path.join(root_dir, 'images')
        self.mask_folder = os.path.join(root_dir, 'masks')

        self.images = os.listdir(self.image_folder)
        self.masks = os.listdir(self.mask_folder)

        assert len(self.images) == len(self.masks), "Number of images and masks should be the same."

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_folder, self.images[idx])
        if "patient" in self.images[idx]:
            mask_path = os.path.join(self.mask_folder, "segmentation_" + self.images[idx][-7:])
        else:
            mask_path = os.path.join(self.mask_folder, "target_seg_" + self.images[idx][-7:])
        
        # Load images
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path)
        
        if self.transform:
            transformed = self.transform(image=image, mask=mask)
            image = transformed['image']
            mask = transformed['mask']

        # Convert to tensors 
        tensor_image = torch.from_numpy(image)
        tensor_image = tensor_image.permute(2, 0, 1)

        tensor_mask = torch.from_numpy(mask)
        tensor_mask = tensor_mask.permute(2, 0, 1) / 255
        tensor_mask = tensor_mask[2:, :, :]
        
        # add padding
        pad_height = max(self.target_size[0] - tensor_image.size(1), 0)
        pad_width = max(self.target_size[1] - tensor_image.size(2), 0)

        pad_top = pad_height // 2
        pad_bottom = pad_height - pad_top
        pad_left = pad_width // 2
        pad_right = pad_width - pad_left

        padded_image = transforms.functional.pad(tensor_image, (pad_left, pad_bottom, pad_right, pad_top), fill=255)
        padded_mask = transforms.functional.pad(tensor_mask, (pad_left, pad_bottom, pad_right, pad_top), fill=0)

        return torch.tensor(padded_image, dtype=torch.float32), torch.tensor(padded_mask, dtype=torch.float32)


In [5]:
model = model.to(device)

# Set up dataset and dataloader
transform = A.Compose([])

#transform = A.Compose([A.GaussNoise(p = 0.2), A.RandomGamma(p = 0.2), A.Sharpen(p=0.2),
#                       A.Resize(width = np.random.randint(200, 416), height = np.random.randint(200,992), p = 0.2)])
""" 


"""
# ALL Transforms
# A.CLAHE(p=0.2)
# A.HorizontalFlip(p=0.5),
# A.RandomGamma(p=0.2)
# A.GridDistortion(p=0.2)
# A.RandomBrightnessContrast(p=0.2)
# A.Resize(width = np.random.randint(200, 416), height = np.random.randint(200,992), p = 0.2)
# A.OneOf([ ],p=0.9) for more 
# A.Sharpen(p=0.2)
# A.Blur(p=0.2)
# A.RandomCrop(height = 200, width=200, p=0.2)
# GaussNoise(p = 0.2)

trainDataset = SegmentationDataset(root_dir=TRAIN_DATA_DIR, transform=transform)
valDataset = SegmentationDataset(root_dir=VAL_DATA_DIR)

train_loader = DataLoader(trainDataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(valDataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)


In [ ]:
# Train Loop
for epoch in range(EPOCHS):
    train_loss = torch.tensor(0.0)
    model.train()

    # Use tqdm to add a progress bar
    for images, masks in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{EPOCHS}', leave=False):
        images, masks = images.to(device), masks.to(device)

        # Forward pass
        outputs = model(images)
        # outputs = torch.argmax(outputs, dim=1).unsqueeze(1).float()

        loss = criterion(outputs, masks)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.detach().cpu()
        optimizer.step()

    train_loss /= len(train_loader)
    run["loss/train_loss"].log(train_loss)

    # Validation
    model.eval()

    val_loss = torch.tensor(0.0)
    iou = torch.tensor(0.0)
    dice_score = torch.tensor(0.0)

    with torch.no_grad():
        for val_images, val_masks in tqdm(val_loader, desc=f'Validation', leave=False):
            val_images, val_masks = val_images.to(device), val_masks.to(device)

            model_outputs = model(val_images)

            val_loss += criterion(model_outputs, val_masks).cpu()

            val_masks_int = torch.tensor(val_masks, dtype=torch.int8)
            dice_score += calc_dice_score(torch.sigmoid(model_outputs), val_masks_int, ignore_index=0).cpu()

            iou += calc_iou(model_outputs, val_masks_int).cpu()


    val_loss /= len(val_loader)
    iou /= len(val_loader)
    dice_score /= len(val_loader)

    if torch.isnan(iou):
        iou = torch.tensor(0.0)

    run["loss/val_loss"].log(val_loss)
    run["val/iou"].log(iou)
    run["val/dice_score"].log(dice_score)

    torch.save(model.state_dict(), save_path + run_name + "_EPOCH_" + str(epoch) + '.pth')

    print(f"Epoch [{epoch + 1}/{EPOCHS}], Train Loss: {train_loss}, Validation Loss: {val_loss}\n"
          f"IOU: {iou}, Dice Score: {dice_score}")

# Save the trained model
torch.save(model.state_dict(), save_path + run_name + "_FINAL" + '.pth')
run[f"network/network_weights"].upload(File(run_name + '.pth'))

run.stop()


Epoch 1/110:   0%|          | 0/42 [00:00<?, ?it/s]/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, dtype=torch.float32), torch.tensor(padded_mask, dtype=torch.float32)
/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, dtype=torch.float32), torch.tensor(padded_mask, dtype=torch.float32)
/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, d

Epoch [1/110], Train Loss: 1.8247190713882446, Validation Loss: 1.7104558944702148
IOU: 0.01065882109105587, Dice Score: 0.02095317840576172


Epoch 2/110:   0%|          | 0/42 [00:00<?, ?it/s]/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, dtype=torch.float32), torch.tensor(padded_mask, dtype=torch.float32)
/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, dtype=torch.float32), torch.tensor(padded_mask, dtype=torch.float32)
/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, d

Epoch [2/110], Train Loss: 1.6140100955963135, Validation Loss: 1.5243184566497803
IOU: 0.020627550780773163, Dice Score: 0.03998257964849472


Epoch 3/110:   0%|          | 0/42 [00:00<?, ?it/s]/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, dtype=torch.float32), torch.tensor(padded_mask, dtype=torch.float32)
/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, dtype=torch.float32), torch.tensor(padded_mask, dtype=torch.float32)
/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, d

...

Epoch 42/110:   0%|          | 0/42 [00:00<?, ?it/s]/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, dtype=torch.float32), torch.tensor(padded_mask, dtype=torch.float32)
/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, dtype=torch.float32), torch.tensor(padded_mask, dtype=torch.float32)
/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, 

Epoch [42/110], Train Loss: 0.9822422862052917, Validation Loss: 1.00824773311615
IOU: 0.4160084128379822, Dice Score: 0.5549716353416443


Epoch 43/110:   0%|          | 0/42 [00:00<?, ?it/s]/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, dtype=torch.float32), torch.tensor(padded_mask, dtype=torch.float32)
/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, dtype=torch.float32), torch.tensor(padded_mask, dtype=torch.float32)
/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, 

Epoch [43/110], Train Loss: 0.9704117774963379, Validation Loss: 1.0015987157821655
IOU: 0.0, Dice Score: 0.5896446704864502


Epoch 44/110:   0%|          | 0/42 [00:00<?, ?it/s]/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, dtype=torch.float32), torch.tensor(padded_mask, dtype=torch.float32)
/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, dtype=torch.float32), torch.tensor(padded_mask, dtype=torch.float32)
/tmp/ipykernel_21830/850540683.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_image, 

In [ ]:
"""
# Declare an augmentation pipeline
transform = A.Compose([
    A.RandomCrop(height = 200, width=200, p=1)
])

image_path = "/notebooks/image_segmentation/network/image_data/train/images/patient_116.png"
mask_path = "/notebooks/image_segmentation/network/image_data/train/masks/segmentation_116.png" 

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
mask = cv2.imread(mask_path)

transformed = transform(image=image, mask=mask)
transformed_image = transformed['image']
transformed_mask = transformed['mask']

image_torch, mask_torch = convert_to_torch(image, mask)
plot_image_and_mask(image_torch, mask_torch)
    
image_torch, mask_torch = convert_to_torch(transformed_image, transformed_mask)
plot_image_and_mask(image_torch, mask_torch)
"""

In [7]:
def plot_image_and_mask(image, mask): 
    image = image/ 255.0
    mask = mask

    # Plot side by side with the mask and mask overlain
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Plot the original image
    axes[0].imshow(image.permute(1, 2, 0))
    axes[0].set_title('Original Image')

    # Plot the mask
    axes[1].imshow(mask.permute(1, 2, 0), cmap='viridis')
    axes[1].set_title('Mask')

    # Overlay the mask on the image
    axes[2].imshow(image.permute(1, 2, 0))
    axes[2].imshow(mask.permute(1, 2, 0), cmap='viridis', alpha=0.6)  # Set alpha to less than 1
    axes[2].set_title('Mask Overlain on Image')

    # Display the plots
    plt.show()

def convert_to_torch(image, mask):
    
    tensor_image = torch.from_numpy(image)
    tensor_image = tensor_image.permute(2,0, 1)

    tensor_mask = torch.from_numpy(mask)
    tensor_mask = tensor_mask.permute(2,0, 1) / 255
    
    tensor_mask = tensor_mask[2:, :, :]

    # add padding
    pad_height = max(992 - tensor_image.size(1), 0)
    pad_width = max(416 - tensor_image.size(2), 0)

    pad_top = pad_height // 2
    pad_bottom = pad_height - pad_top
    pad_left = pad_width // 2
    pad_right = pad_width - pad_left

    padded_image = transforms.functional.pad(tensor_image, (pad_left, pad_bottom, pad_right, pad_top), fill=255)
    padded_mask = transforms.functional.pad(tensor_mask, (pad_left, pad_bottom, pad_right, pad_top), fill=0)


    return torch.tensor(padded_image, dtype=torch.float32), torch.tensor(padded_mask, dtype=torch.float32)

In [8]:
image_path = "/notebooks/image_segmentation/network/image_data/train/images/patient_116.png"
mask_path = "/notebooks/image_segmentation/network/image_data/train/masks/segmentation_116.png" 
model_path = "/notebooks/Testing/CV-131/MODEL-UnetPlusPlusresnet34CV-131_FINAL.pth" 

model = smp.UnetPlusPlus(
    encoder_name=ENCODER_NAME,           # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    in_channels=3,                        # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                            # model output channels (number of classes in your dataset)
)
model.load_state_dict(torch.load(model_path))
model.eval()

def predict_image(model, image, mask): 

    if type(image) == str: 
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path)
    
        image, mask = convert_to_torch(image, mask)
    
    out = model(image.unsqueeze(0)) 

    segment_map = torch.sigmoid(out).squeeze(0).detach()
    segment_map = (segment_map > 0.5) * 1
    
    plot_image_and_mask(image, mask)
    plot_image_and_mask(image, segment_map)
    
#predict_image(model, image_path, mask_path)

RuntimeError: Error(s) in loading state_dict for UnetPlusPlus:
	Missing key(s) in state_dict: "encoder.stem.conv.weight", "encoder.stem.bn.weight", "encoder.stem.bn.bias", "encoder.stem.bn.running_mean", "encoder.stem.bn.running_var", "encoder.s1.b1.conv1.conv.weight", "encoder.s1.b1.conv1.bn.weight", "encoder.s1.b1.conv1.bn.bias", "encoder.s1.b1.conv1.bn.running_mean", "encoder.s1.b1.conv1.bn.running_var", "encoder.s1.b1.conv2.conv.weight", "encoder.s1.b1.conv2.bn.weight", "encoder.s1.b1.conv2.bn.bias", "encoder.s1.b1.conv2.bn.running_mean", "encoder.s1.b1.conv2.bn.running_var", "encoder.s1.b1.conv3.conv.weight", "encoder.s1.b1.conv3.bn.weight", "encoder.s1.b1.conv3.bn.bias", "encoder.s1.b1.conv3.bn.running_mean", "encoder.s1.b1.conv3.bn.running_var", "encoder.s1.b1.downsample.conv.weight", "encoder.s1.b1.downsample.bn.weight", "encoder.s1.b1.downsample.bn.bias", "encoder.s1.b1.downsample.bn.running_mean", "encoder.s1.b1.downsample.bn.running_var", "encoder.s1.b2.conv1.conv.weight", "

In [ ]:
count = 0 

for image, mask in DataLoader(valDataset, batch_size=1, shuffle=False, num_workers=0): 
    predict_image(model, image.squeeze(0), mask.squeeze(0))
    print("-----------------------")
    count += 1
    if count == 50:
        break